# 加载数据

In [1]:
import os
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
from torchvision.transforms import transforms
from torchsummary import summary

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Oct 18 06:22:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip /content/drive/MyDrive/Data.zip


In [ ]:
!ls /content/drive/MyDrive/Data/Data/data/train -l | grep "^-" | wc -l
!ls /content/drive/MyDrive/Data/Data/data/test -l | grep "^-" | wc -l

1534
384


In [ ]:
%cd /content
if os.path.exists("Ultrasound"):
    %rm -rf Ultrasound
!ls
if not os.path.exists("Ultrasound"):
    !git clone https://github.com/z872845991/Ultrasound.git
%cd Ultrasound
!ls

In [2]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# 参数

## 导入工具包

In [3]:
from dataset.Fetus import FetusDataset
from tools.metrics import dice_coef,iou_score,get_accuracy,get_precision,get_specificity,get_recall,get_F1
from tools.utils import AverageMeter
from train.train_unet_local import train_model_local
import matplotlib.pyplot as plt
import datetime

## 导入模型

In [4]:
from model.unet_res_myself_pool_connect_o_C import Unet_res_myself_pool
from model.unet import Unet
from model.idea3 import Idea3
from model.idea3 import Idea3_seblock
from model.unet_self_attention import Unet_self_attention
from model.unet_encoder_idea3_se import Unet_encoder_idea3_se
from model.unet_decoder_se import Unet_decoder_se

In [5]:
num_epoch=81
batch_size={'train':2,
            'val':1
            }
train_path='/content/drive/MyDrive/Data/Data/data/train'
val_path='/content/drive/MyDrive/Data/Data/data/test'
#变换
x_transforms = transforms.Compose([
    transforms.Resize((512,512)),
    # transforms.CenterCrop(512),
    transforms.ToTensor()
    # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
# mask只需要转换为tensor
y_transforms = transforms.Compose([
    transforms.Resize((512, 512)),
    # transforms.CenterCrop(512),
    transforms.ToTensor()
   ])
#加载数据集
train_dataset=FetusDataset(train_path,transform=x_transforms,target_transform=y_transforms)
val_dataset=FetusDataset(val_path,transform=x_transforms,target_transform=y_transforms)
#设置dataloader
dataloaders={
    'train':DataLoader(train_dataset,batch_size=batch_size['train'],shuffle=True,num_workers=0),
    'val':DataLoader(val_dataset,batch_size=batch_size['val'],shuffle=True,num_workers=0)
             }



FileNotFoundError: [WinError 3] 系统找不到指定的路径。: '/content/drive/MyDrive/Data/Data/data/train'

# 训练

## 1. Idea3

In [ ]:
model1=Idea3(n_class=1)
model1.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model1.parameters())

train1=train_model_local(model1)
train1.summarys(input_size=(3,224,224))

In [ ]:
train1.compile(dataloaders,criterion,optimizer,num_epoch,batch_size,train_path,val_path,device)

In [ ]:
file11='/content/drive/MyDrive/result/train/train_7m_Idea3_change.txt'
file12='/content/drive/MyDrive/result/test/test_7m_Idea3_change.txt'
path1='/content/drive/MyDrive/result/checkpoins/train_7m_Idea3'
train1.fit(file11,file12,path1)

In [ ]:
path1='/content/drive/MyDrive/result/checkpoints/train_7m_Idea3_se_change_1'
with torch.no_grad():
    data=next(iter(dataloaders['val']))
    pic1,label=data[0].to(device),data[1].to(device)
    path1=path1+'_%d.pth'%(num_epoch-1)
    model1.load_state_dict(torch.load(path1))
    y=model1(pic1).cpu()
    plt.figure(1)
    plt.subplot(121)
    img1_y=torch.squeeze(y).numpy()
    plt.imshow(img1_y,cmap='gray')
    plt.subplot(122)
    img_label=label.squeeze().cpu().numpy()
    plt.imshow(img_label) 

## 2. Unet_res_myself_pool


In [ ]:
model2=Unet_res_myself_pool(n_class=1)
#model=Unet(n_class=1)
model2.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model2.parameters())

train2=train_model_local(model2)
train2.summarys(input_size=(3,224,224))

In [ ]:
train2.compile(dataloaders,criterion,optimizer,num_epoch,batch_size,train_path,val_path,device)

In [ ]:
file21='/content/drive/MyDrive/result/train/train_7m_Unet_res_myself_pool_change.txt'
file22='/content/drive/MyDrive/result/test/test_7m_Unet_res_myself_pool_change.txt'
path2='/content/drive/MyDrive/result/checkpoints/train_7m_Unet_res_myself_pool_change'
train2.fit(file21,file22,path2)

In [ ]:
path2='/content/drive/MyDrive/result/checkpoints/train_7m_Idea3_se_change_2'
path2=path2+'_%d.pth'%(num_epoch-1)
with torch.no_grad():
    data=next(iter(dataloaders['val']))
    pic2,label=data[0].to(device),data[2].to(device)

    model2.load_state_dict(torch.load(path2))
    
    y=model2(pic2).cpu()
    plt.figure(1)
    plt.subplot(121)
    img2_y=torch.squeeze(y).numpy()
    plt.imshow(img2_y,cmap='gray')
    plt.subplot(122)
    img_label=label.squeeze().cpu().numpy()
    plt.imshow(img_label) 

## 3. Unet


In [ ]:
model3=Unet(n_class=1)
model3.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model3.parameters())

train3=train_model_local(model3)
train3.summarys(input_size=(3,224,224))

In [ ]:
train3.compile(dataloaders,criterion,optimizer,num_epoch,batch_size,train_path,val_path,device)

In [ ]:
file31='/content/drive/MyDrive/result/train/train_7m_Unet_change.txt'
file32='/content/drive/MyDrive/result/test/test_7m_Unet_change.txt'
path3='/content/drive/MyDrive/result/checkpoints/train_7m_Unet_change'
train3.fit(file31,file32,path3)

In [ ]:
path3='/content/drive/MyDrive/result/checkpoints/train_7m_Idea3_se_change_1'
path3=path3+'_%d.pth'%(num_epoch-1)
with torch.no_grad():
    data=next(iter(dataloaders['val']))
    pic3,label=data[0].to(device),data[1].to(device)
    model3.load_state_dict(torch.load(path3))
    y=model3(pic3).cpu()
    plt.figure(3)
    plt.subplot(121)
    img3_y=torch.squeeze(y).numpy()
    plt.imshow(img3_y,cmap='gray')
    plt.subplot(122)
    img_label=label.squeeze().cpu().numpy()
    plt.imshow(img_label) 

## 4. Unet_self_attention

In [ ]:
model4=Unet_self_attention(n_class=1)
model4.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model4.parameters())

train4=train_model_local(model4)
train4.summarys(input_size=(3,224,224))

In [ ]:
train4.compile(dataloaders,criterion,optimizer,num_epoch,batch_size,train_path,val_path,device)

In [ ]:
start=datetime.datetime.now()
file41='/content/drive/MyDrive/result/train/train_7m_Unet_self_attention_.txt'
file42='/content/drive/MyDrive/result/test/test_7m_Unet_self_attention.txt'
path4='/content/drive/MyDrive/result/checkpoints/train_7m_Unet_self_attention'
train4.fit(file41,file42,path4)
end=datetime.datetime.now()
t=end-start
print("%d min, %d seconds",%(t/60,t%60))

In [ ]:
path4='/content/drive/MyDrive/result/checkpoints/train_7m_Unet_self_attention'
path4=path4+'_%d.pth'%(num_epoch-1)
with torch.no_grad():
    data=next(iter(dataloaders['val']))

    pic4,label=data[0].to(device),data[1].to(device)
    model4.load_state_dict(torch.load(path3))
    y=model4(pic4).cpu()
    
    plt.figure(3)
    plt.subplot(121)
    img4_y=torch.squeeze(y).numpy()
    plt.imshow(img4_y,cmap='gray')
    plt.subplot(122)
    img_label=label.squeeze().cpu().numpy()
    plt.imshow(img_label) 

## 5. Unet_encoder_idea3_se

In [6]:
model5=Unet_encoder_idea3_se(n_class=1)
model5.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model5.parameters())

train5=train_model_local(model5)
train5.summarys(input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
         MaxPool2d-7         [-1, 64, 112, 112]               0
            Conv2d-8         [-1, 64, 112, 112]           4,160
       BatchNorm2d-9         [-1, 64, 112, 112]             128
             ReLU-10         [-1, 64, 112, 112]               0
           Conv2d-11         [-1, 64, 112, 112]          36,928
      BatchNorm2d-12         [-1, 64, 112, 112]             128
             ReLU-13         [-1, 64, 112, 112]               0
           Conv2d-14         [-1, 64, 1

## 6.unet_decoder_idea3_se

In [ ]:
model6=Unet_decoder_idea3_se(n_class=1)
model6.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model6.parameters())

train6=train_model_local(model6)
train6.summarys(input_size=(3,224,224))

In [ ]:
train6.compile(dataloaders,criterion,optimizer,num_epoch,batch_size,train_path,val_path,device)

In [ ]:
start=datetime.datetime.now()
file61='/content/drive/MyDrive/result/train/train_7m_Unet_decoder_idea3_se_change.txt'
file62='/content/drive/MyDrive/result/test/test_7m_Unet_decoder_idea3_se_change.txt'
path6='/content/drive/MyDrive/result/checkpoints/train_7m_Unet_decoder_idea3_se_change'
train6.fit(file61,file62,path6)
end=datetime.datetime.now()
t=(end-start).total_seconds()
print("%d min, %d seconds"%((t/60),(t%60)))

In [ ]:
path6='/content/drive/MyDrive/result/checkpoints/train_7m_Unet_decoder_idea3_se_change'
path6=path6+'_%d.pth'%(num_epoch-1)
with torch.no_grad():
    data=next(iter(dataloaders['val']))
    pic3,label=data[0].to(device),data[1].to(device)

    model6.load_state_dict(torch.load(path6))   #修改model,path
    y=model6(pic3).cpu()   #修改model

    plt.figure(3)
    plt.subplot(121)
    img3_y=torch.squeeze(y).numpy()
    plt.imshow(img3_y,cmap='gray')
    plt.subplot(122)
    img_label=label.squeeze().cpu().numpy()
    plt.imshow(img_label) 

## 7.unet_decoder_se

In [ ]:
model7=Unet_decoder_se(n_class=1)
model7.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model7.parameters(),lr=1e-4)

train7=train_model_local(model6)
train7.summarys(input_size=(3,224,224))

In [ ]:
train7.compile(dataloaders,criterion,optimizer,num_epoch,batch_size,train_path,val_path,device)

In [ ]:
start=datetime.datetime.now()
file71='/content/drive/MyDrive/result/train/train_7m_Unet_decoder_se_change.txt'
file72='/content/drive/MyDrive/result/test/test_7m_Unet_decode_se_change.txt'
path7='/content/drive/MyDrive/result/checkpoints/train_7m_Unet_decoder_se_change'
train7.fit(file71,file72,path7)
end=datetime.datetime.now()
t=(end-start).total_seconds()
print("%d min, %d seconds"%((t/60),(t%60)))

In [ ]:
path7='/content/drive/MyDrive/result/checkpoints/train_7m_Unet_decoder_se_change'
path7=path7+'_%d.pth'%(num_epoch-1)
with torch.no_grad():
    data=next(iter(dataloaders['val']))
    pic3,label=data[0].to(device),data[1].to(device)

    model7.load_state_dict(torch.load(path7))   #修改model,path
    y=model7(pic3).cpu()   #修改model

    plt.figure(3)
    plt.subplot(121)
    img3_y=torch.squeeze(y).numpy()
    plt.imshow(img3_y,cmap='gray')
    plt.subplot(122)
    img_label=label.squeeze().cpu().numpy()
    plt.imshow(img_label) 

## 14.unet_decoder_idea3_sev2

model6=Unet_decoder_idea3_se(n_class=1)
model6.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model6.parameters())

train6=train_model_local(model6)
train6.summarys(input_size=(3,224,224))

train6.compile(dataloaders,criterion,optimizer,num_epoch,batch_size,train_path,val_path,device)

start=datetime.datetime.now()
file61='/content/drive/MyDrive/result/train/train_7m_Unet_decoder_idea3_se_change.txt'
file62='/content/drive/MyDrive/result/test/test_7m_Unet_decoder_idea3_se_change.txt'
path6='/content/drive/MyDrive/result/checkpoints/train_7m_Unet_decoder_idea3_se_change'
train6.fit(file61,file62,path6)
end=datetime.datetime.now()
t=(end-start).total_seconds()
print("%d min, %d seconds"%((t/60),(t%60)))

path6='/content/drive/MyDrive/result/checkpoints/train_7m_Unet_decoder_idea3_se_change'
path6=path6+'_%d.pth'%(num_epoch-1)
with torch.no_grad():
    data=next(iter(dataloaders['val']))
    pic3,label=data[0].to(device),data[1].to(device)

    model6.load_state_dict(torch.load(path6))   #修改model,path
    y=model6(pic3).cpu()   #修改model

    plt.figure(3)
    plt.subplot(121)
    img3_y=torch.squeeze(y).numpy()
    plt.imshow(img3_y,cmap='gray')
    plt.subplot(122)
    img_label=label.squeeze().cpu().numpy()
    plt.imshow(img_label) 

In [1]:
!pip install tqm

ERROR: Could not find a version that satisfies the requirement tqm
ERROR: No matching distribution found for tqm
